## Tiny ImageNet Challenge -  EIP 3.0

---

### There are few utility functions first up to load the data and set up the generators

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! unzip -q '/content/drive/My Drive/tiny-imagenet-200.zip'

In [0]:
%cd /content/drive/My\ Drive/Assignment\ 4/

/content/drive/My Drive/Assignment 4


In [0]:
import os
import cv2
import imutils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.models import load_model
import keras.backend as K
import json
import sys


TRAIN = "/content/tiny-imagenet-200/train/"
VAL = "/content/tiny-imagenet-200/val/images"
VAL_ANNOT = "/content/tiny-imagenet-200/val/val_annotations.txt"
WORDNET = "/content/tiny-imagenet-200/wnits.txt"
WORD_LABELS = "/content/tiny-imagenet-200/words.txt"

Using TensorFlow backend.


## Used 10 types of image augmentations as specified in the comments

---

1. Rotation Angle
2. Zoom Range
3. Width Shift
- Height Shift
- Shear Range
-  Horizontal Flip
-  Fills empty with reflections
- Increasing/decreasing brightness
-  Pepper augmentation (The size percent makes it similar to Cutout/Occlusion)
- CoarseSaltAndPepper augmentations (imgaug)

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from imgaug import augmenters as iaa

val_data = pd.read_csv(VAL_ANNOT , sep='\t', names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X','Y','H', 'W'], axis=1, inplace=True)

# Augmentations to be include: Increased Contrast, Random Erasing/Cutout/Occlusion, SalandPepper Noise (Prevents Overfitting)
seq = iaa.Sequential([iaa.Pepper(0.02), iaa.CoarseSaltAndPepper(size_percent=0.1)])


train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=18, # Rotation Angle
        zoom_range=0.15,  # Zoom Range
        width_shift_range=0.2, # Width Shift
        height_shift_range=0.2, # Height Shift
        shear_range=0.15,  # Shear Range
        horizontal_flip=True, # Horizontal Flip
        fill_mode="reflect", # Fills empty with reflections
        brightness_range=[0.4, 1.6],  # Increasing/decreasing brightness
        preprocessing_function=(seq.augment_image)   # Pepper and CoarseSaltAndPepper augmentations
)

train_generator = train_datagen.flow_from_directory(
        TRAIN,
        target_size=(64, 64),
        batch_size=128,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_dataframe(
    val_data, directory=VAL, 
    x_col='File', 
    y_col='Class', 
    target_size=(64, 64),
    color_mode='rgb', 
    class_mode='categorical', 
    batch_size=128, 
    shuffle=False, 
    seed=42
)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


## Custom layered ResNet that uses Pre-Activated Layers and BottleNeck Blocks with SeparableConv2D

---


-  The reason to use 1x1 to increase the number of channels is to create a wider model with minimum increase in trainable parameters
- I initially replaced stride by 2 in the shortcut connections with AveragePooling2D but saw a performance drop and so reverted back (Trained from scratch again!)
- Uses SeparableConv2D rather than vanilla Conv2D and this drastically reduced the number of parameters

In [0]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.layers.convolutional import AveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation
from CyclicLearningRate.clr_callback import *
from keras.layers import Flatten, Input, add
from keras.optimizers import Adam
from keras.callbacks import *
from keras.models import Model
from keras.regularizers import l2
from keras import backend as K

class ResNet:
	@staticmethod
	def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
		shortcut = data

		bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
		act1 = Activation("relu")(bn1)
		conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act1)

		bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
		act2 = Activation("relu")(bn2)
		conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, kernel_regularizer=l2(reg))(act2)

		bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
		act3 = Activation("relu")(bn3)
		conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

		if red:
			shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

		x = add([conv3, shortcut])

		return x

	@staticmethod
	def build(width, height, depth, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
		inputShape = (height, width, depth)
		chanDim = -1

		inputs = Input(shape=inputShape)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(inputs)

		x = Conv2D(filters[0], (5, 5), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = ZeroPadding2D((1, 1))(x)
		x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
		for i in range(0, len(stages)):
			stride = (1, 1) if i == 0 else (2, 2)
			x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

			for j in range(0, stages[i] - 1):
				x = ResNet.residual_module(x, filters[i + 1], (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = AveragePooling2D((8, 8))(x)
  
		x = Conv2D(200, (1,1), kernel_regularizer=l2(reg))(x)
		x = Flatten()(x)
		x = Activation("softmax")(x)

		model = Model(inputs, x, name="resnet")

		return model





In [0]:
model = ResNet.build(64, 64, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)

Instructions for updating:
Colocations handled automatically by placer.


## Custom callback to save after every 5 epochs

In [0]:
from keras.callbacks import Callback

class EpochCheckpoint(Callback):
	def __init__(self, outputPath, every=5, startAt=0):
		# call the parent constructor
		super(Callback, self).__init__()
    
		self.outputPath = outputPath
		self.every = every
		self.intEpoch = startAt

	def on_epoch_end(self, epoch, logs={}):
		# check to see if the model should be serialized to disk
		if (self.intEpoch + 1) % self.every == 0:
			p = os.path.sep.join([self.outputPath,
				"resnet.hdf5".format(self.intEpoch + 1)])
			self.model.save(p, overwrite=True)
		self.intEpoch += 1


## Custom Learning Rate Decay callback


---


- Used to train smaller batch sizes and decaying LR to find a better convergence

In [0]:
# Learning Rate Decay

from keras.callbacks import LearningRateScheduler

NUM_EPOCHS = 35
INIT_LR = 0.0000002

def poly_decay(epoch):
  maxEpochs = NUM_EPOCHS
  baseLR = INIT_LR
  power = 1.0
  
  # compute the new learning rate based on polynomial decay
  alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
  
  # return the new learning rate
  return alpha

## Cyclic Learning Rate to train larger batch sizes for the first 100 Epochs


In [0]:
import tensorflow
flag = 0

checkpoint_path = "../checkpoints/check.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback

cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=False,
                                                 verbose=1, period=5)
# Cyclic Learning Rate
clr_triangular = CyclicLR(mode='triangular', max_lr=0.1)


callbacks = [
  EpochCheckpoint('../checkpoints/', every=1),
   cp_callback,
    LearningRateScheduler(poly_decay)   
]
# clr_triangular

## Model Summary (Params much lesser than 26M)

---


### Total params: 2,265,748
### Trainable params: 2,252,046
### Non-trainable params: 13,702

In [0]:
from keras.models import load_model
from keras.optimizers import SGD

if flag == 0: 
  model = ResNet.build(64, 64, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
  opt = SGD(lr=INIT_LR, momentum=0.9)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#   model.compile(optimizer=Adam(0.1), loss="categorical_crossentropy", metrics=["accuracy"])

else:
  
  # Load the model
  model = load_model('../checkpoints/resnet.hdf5')
  
#   Update the learning rate
  print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
  K.set_value(model.optimizer.lr, le-3)
  print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   4800        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 64, 64, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation

## Attempted to train 50 epochs but an error occurred after 35 epochs

In [0]:
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=50,
  max_queue_size=64 * 2,
  callbacks=callbacks, verbose=1
)

# close the databases
train_gen.close()
val_gen.close()

Epoch 1/50
1562/1562 [==============================] - 625s 400ms/step - loss: 5.2895 - acc: 0.0522 - val_loss: 4.9462 - val_acc: 0.0768
Epoch 2/50
1562/1562 [==============================] - 595s 381ms/step - loss: 4.5602 - acc: 0.1253 - val_loss: 5.1685 - val_acc: 0.1002
Epoch 3/50
1562/1562 [==============================] - 597s 382ms/step - loss: 4.2459 - acc: 0.1721 - val_loss: 4.4953 - val_acc: 0.1498
Epoch 4/50
1562/1562 [==============================] - 588s 376ms/step - loss: 4.0724 - acc: 0.2053 - val_loss: 4.4285 - val_acc: 0.1740
Epoch 5/50
1562/1562 [==============================] - 562s 360ms/step - loss: 3.9622 - acc: 0.2322 - val_loss: 4.6980 - val_acc: 0.1520

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/50
1562/1562 [==============================] - 530s 339ms/step - loss: 3.8921 - acc: 0.2498 - val_loss: 4.1909 - val_acc: 0.2251
Epoch 7/50
1562/1562 [==============================] - 549s 351ms/step - loss: 3.8325 - acc: 0.2693 - val_loss: 4.3

## There was a Buffered data error and thus had to stop the execution after 38 epochs and the model was saved after 35

---


## Total Epoch count: 35

In [0]:
from keras.models import load_model
from keras.optimizers import SGD
flag = 1

if flag == 0: 
  model = ResNet.build(64, 64, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
  opt = SGD(lr=INIT_LR, momentum=0.9)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#   model.compile(optimizer=Adam(0.1), loss="categorical_crossentropy", metrics=["accuracy"])

else:
  
  # Load the model
  model = load_model(checkpoint_path)
  
#   Update the learning rate
  print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
  K.set_value(model.optimizer.lr, 0.1)
  print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=40,
  max_queue_size=64 * 2,
  callbacks=callbacks, verbose=1
)

model.save(checkpoint_path)

Old Learning Rate: 0.08666666597127914
New Learning Rate: 0.10000000149011612
Epoch 1/40
1562/1562 [==============================] - 603s 386ms/step - loss: 3.6570 - acc: 0.3987 - val_loss: 4.1770 - val_acc: 0.3471
Epoch 2/40
1562/1562 [==============================] - 585s 374ms/step - loss: 3.6475 - acc: 0.4019 - val_loss: 3.9846 - val_acc: 0.3555
Epoch 3/40
1562/1562 [==============================] - 585s 375ms/step - loss: 3.6283 - acc: 0.4067 - val_loss: 4.0775 - val_acc: 0.3477
Epoch 4/40
1562/1562 [==============================] - 587s 376ms/step - loss: 3.6121 - acc: 0.4094 - val_loss: 4.1697 - val_acc: 0.3381
Epoch 5/40
1562/1562 [==============================] - 587s 376ms/step - loss: 3.5915 - acc: 0.4115 - val_loss: 4.0687 - val_acc: 0.3538

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/40
1562/1562 [==============================] - 553s 354ms/step - loss: 3.5840 - acc: 0.4127 - val_loss: 3.8007 - val_acc: 0.3833
Epoch 7/40
1562/1562 [================

## Colab disconnected after 28 epochs and was saved at 25 epochs

---


## Total epochs = 60

In [0]:
from keras.models import load_model
from keras.optimizers import SGD
flag = 1

if flag == 0: 
  model = ResNet.build(64, 64, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
  opt = SGD(lr=INIT_LR, momentum=0.9)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
#   model.compile(optimizer=Adam(0.1), loss="categorical_crossentropy", metrics=["accuracy"])

else:
  
  # Load the model
  model = load_model(checkpoint_path)
  
#   Update the learning rate
  print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
  K.set_value(model.optimizer.lr, 0.005)
  print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=64 * 2,
  callbacks=callbacks, verbose=1
)

model.save(checkpoint_path)

Old Learning Rate: 0.04800000041723251
New Learning Rate: 0.004999999888241291
Epoch 1/35
1562/1562 [==============================] - 565s 362ms/step - loss: 3.6756 - acc: 0.4125 - val_loss: 3.9539 - val_acc: 0.3959
Epoch 2/35
1562/1562 [==============================] - 553s 354ms/step - loss: 3.6580 - acc: 0.4309 - val_loss: 3.7107 - val_acc: 0.4328
Epoch 3/35
1562/1562 [==============================] - 552s 354ms/step - loss: 3.6197 - acc: 0.4363 - val_loss: 3.8290 - val_acc: 0.4160
Epoch 4/35
1562/1562 [==============================] - 553s 354ms/step - loss: 3.6019 - acc: 0.4382 - val_loss: 3.8936 - val_acc: 0.3996
Epoch 5/35
1562/1562 [==============================] - 555s 355ms/step - loss: 3.5804 - acc: 0.4405 - val_loss: 5.0070 - val_acc: 0.2900

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
1562/1562 [==============================] - 522s 334ms/step - loss: 3.5633 - acc: 0.4413 - val_loss: 4.3363 - val_acc: 0.3408
Epoch 7/35
1562/1562 [===============

## Another 30 epochs where model was saved

---


## Total epochs = 90

In [0]:
from keras.models import load_model
from keras.optimizers import SGD
flag = 1

if flag == 0: 
  model = ResNet.build(64, 64, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
  opt = SGD(lr=INIT_LR, momentum=0.9)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

else:
  
  # Load the model
  model = load_model(checkpoint_path)

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=64 * 2,
  callbacks=callbacks, verbose=1
)

model.save(checkpoint_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/35
1562/1562 [==============================] - 580s 371ms/step - loss: 3.6153 - acc: 0.4358 - val_loss: 4.1227 - val_acc: 0.3787
Epoch 2/35
1562/1562 [==============================] - 557s 356ms/step - loss: 3.6074 - acc: 0.4396 - val_loss: 4.2174 - val_acc: 0.3608
Epoch 3/35
1562/1562 [==============================] - 552s 354ms/step - loss: 3.5930 - acc: 0.4424 - val_loss: 4.2454 - val_acc: 0.3555
Epoch 4/35
1562/1562 [==============================] - 544s 348ms/step - loss: 3.5772 - acc: 0.4428 - val_loss: 3.7043 - val_acc: 0.4287
Epoch 5/35
1562/1562 [==============================] - 535s 343ms/step - loss: 3.5520 - acc: 0.4472 - val_loss: 4.2623 - val_acc: 0.3523

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
1562/1562 [==============================] - 518s 332ms/step - loss: 3.5438 - acc: 0.4466 - val_loss: 3.9182 - val_acc: 0.4

## Trained for another 35 epochs and noticed the variability of the data. Therefore decided to further reduce the learning rate and stop Cyclic Learning Rate and replace with Learning Rate Decay


---

##  Total Epochs = 125

In [0]:
K.set_value(model.optimizer.momentum, 0.9)
print(K.get_value(model.optimizer.momentum))

model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=64 * 2,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')
model.save(checkpoint_path)

0.9
Epoch 1/35
1562/1562 [==============================] - 563s 360ms/step - loss: 3.6252 - acc: 0.4309 - val_loss: 4.4610 - val_acc: 0.3353
Epoch 2/35
1562/1562 [==============================] - 561s 359ms/step - loss: 3.6410 - acc: 0.4422 - val_loss: 4.3583 - val_acc: 0.3448
Epoch 3/35
1562/1562 [==============================] - 568s 364ms/step - loss: 3.6180 - acc: 0.4436 - val_loss: 4.2812 - val_acc: 0.3707
Epoch 4/35
1562/1562 [==============================] - 563s 361ms/step - loss: 3.5863 - acc: 0.4489 - val_loss: 3.9269 - val_acc: 0.4130
Epoch 5/35
1562/1562 [==============================] - 569s 364ms/step - loss: 3.5695 - acc: 0.4510 - val_loss: 4.1916 - val_acc: 0.3697

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
1562/1562 [==============================] - 539s 345ms/step - loss: 3.5513 - acc: 0.4513 - val_loss: 4.4334 - val_acc: 0.3386
Epoch 7/35
1562/1562 [==============================] - 568s 364ms/step - loss: 3.5440 - acc: 0.4524 - val_loss:

## The model training has been very slow. Thus, I've increased the batch size and changed to CLR from Learning Rate Decay

---

## Total Epochs = 160

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 128,
  validation_data=val_generator,
  validation_steps=10000 // 128,
  epochs=35,
  max_queue_size=128 * 2,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')
model.save(checkpoint_path)

Epoch 1/35
  2/781 [..............................] - ETA: 1:24:55 - loss: 2.9759 - acc: 0.5352

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.572852). Check your callbacks.
  % delta_t_median)


781/781 [==============================] - 415s 531ms/step - loss: 2.5838 - acc: 0.6194 - val_loss: 3.0819 - val_acc: 0.5332
Epoch 2/35
781/781 [==============================] - 384s 491ms/step - loss: 2.5295 - acc: 0.6106 - val_loss: 3.5392 - val_acc: 0.4467
Epoch 3/35
781/781 [==============================] - 383s 491ms/step - loss: 2.8241 - acc: 0.5469 - val_loss: 3.5283 - val_acc: 0.4502
Epoch 4/35
781/781 [==============================] - 385s 493ms/step - loss: 2.6342 - acc: 0.5870 - val_loss: 3.1412 - val_acc: 0.5056
Epoch 5/35
781/781 [==============================] - 386s 494ms/step - loss: 2.2284 - acc: 0.6655 - val_loss: 2.8317 - val_acc: 0.5506

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
781/781 [==============================] - 370s 474ms/step - loss: 2.0306 - acc: 0.7058 - val_loss: 3.0236 - val_acc: 0.5145
Epoch 7/35
781/781 [==============================] - 370s 474ms/step - loss: 2.3653 - acc: 0.6195 - val_loss: 3.3761 - val_acc: 0.4444
Epo

## Only 5 epochs were saved! 

---

## Total Epochs = 165

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 128,
  validation_data=val_generator,
  validation_steps=10000 // 128,
  epochs=35,
  max_queue_size=128 * 2,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')
model.save(checkpoint_path)

Epoch 1/35
781/781 [==============================] - 430s 550ms/step - loss: 2.0517 - acc: 0.7000 - val_loss: 2.9504 - val_acc: 0.5201
Epoch 2/35
781/781 [==============================] - 402s 515ms/step - loss: 2.4252 - acc: 0.6073 - val_loss: 3.5299 - val_acc: 0.4311
Epoch 3/35
781/781 [==============================] - 410s 525ms/step - loss: 2.8444 - acc: 0.5341 - val_loss: 3.4779 - val_acc: 0.4537
Epoch 4/35
781/781 [==============================] - 410s 525ms/step - loss: 2.6202 - acc: 0.5883 - val_loss: 3.3111 - val_acc: 0.4709
Epoch 5/35
781/781 [==============================] - 411s 526ms/step - loss: 2.1995 - acc: 0.6718 - val_loss: 2.8467 - val_acc: 0.5510

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
781/781 [==============================] - 376s 481ms/step - loss: 2.0025 - acc: 0.7122 - val_loss: 3.1268 - val_acc: 0.4958
Epoch 7/35
781/781 [==============================] - 407s 522ms/step - loss: 2.3647 - acc: 0.6219 - val_loss: 3.1118 - val_acc:

## Total Epochs = 200

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=35,
  max_queue_size=128 * 4,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')
model.save(checkpoint_path)

Instructions for updating:
Use tf.cast instead.
Epoch 1/35
390/390 [==============================] - 394s 1s/step - loss: 2.2497 - acc: 0.6872 - val_loss: 3.0517 - val_acc: 0.5413
Epoch 2/35
390/390 [==============================] - 325s 833ms/step - loss: 2.0525 - acc: 0.7284 - val_loss: 3.0347 - val_acc: 0.5453
Epoch 3/35
390/390 [==============================] - 326s 835ms/step - loss: 2.0216 - acc: 0.7221 - val_loss: 2.9802 - val_acc: 0.5394
Epoch 4/35
390/390 [==============================] - 326s 836ms/step - loss: 2.0745 - acc: 0.6966 - val_loss: 3.1678 - val_acc: 0.5073
Epoch 5/35
390/390 [==============================] - 326s 836ms/step - loss: 2.2135 - acc: 0.6592 - val_loss: 3.3584 - val_acc: 0.4680

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
390/390 [==============================] - 314s 805ms/step - loss: 2.2881 - acc: 0.6443 - val_loss: 3.0881 - val_acc: 0.5142
Epoch 7/35
390/390 [==============================] - 318s 816ms/step - loss: 2.135

## Total epochs = 235

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=35,
  max_queue_size=128 * 4,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')
model.save(checkpoint_path)

Epoch 1/35
390/390 [==============================] - 370s 950ms/step - loss: 1.9121 - acc: 0.7196 - val_loss: 2.9562 - val_acc: 0.5369
Epoch 2/35
390/390 [==============================] - 311s 797ms/step - loss: 1.6409 - acc: 0.7804 - val_loss: 2.9299 - val_acc: 0.5489
Epoch 3/35
390/390 [==============================] - 310s 794ms/step - loss: 1.7659 - acc: 0.7413 - val_loss: 3.3198 - val_acc: 0.4808
Epoch 4/35
390/390 [==============================] - 310s 794ms/step - loss: 2.0254 - acc: 0.6795 - val_loss: 3.7144 - val_acc: 0.4312
Epoch 5/35
390/390 [==============================] - 309s 793ms/step - loss: 2.2597 - acc: 0.6370 - val_loss: 3.9806 - val_acc: 0.3922

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
390/390 [==============================] - 310s 795ms/step - loss: 2.3488 - acc: 0.6288 - val_loss: 3.3549 - val_acc: 0.4725
Epoch 7/35
390/390 [==============================] - 308s 791ms/step - loss: 2.1537 - acc: 0.6766 - val_loss: 3.0887 - val_acc:

## Total Epochs = 270

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=35,
  max_queue_size=128 * 4,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')
model.save(checkpoint_path)

Epoch 1/35
  2/390 [..............................] - ETA: 42:06 - loss: 2.4083 - acc: 0.6172  

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.801945). Check your callbacks.
  % delta_t_median)


390/390 [==============================] - 389s 997ms/step - loss: 1.9558 - acc: 0.7180 - val_loss: 2.9219 - val_acc: 0.5592
Epoch 2/35
390/390 [==============================] - 324s 831ms/step - loss: 1.6414 - acc: 0.7905 - val_loss: 3.1314 - val_acc: 0.5303
Epoch 3/35
390/390 [==============================] - 327s 838ms/step - loss: 1.7588 - acc: 0.7549 - val_loss: 3.4664 - val_acc: 0.4658
Epoch 4/35
390/390 [==============================] - 330s 845ms/step - loss: 2.0339 - acc: 0.6872 - val_loss: 3.5265 - val_acc: 0.4595
Epoch 5/35
390/390 [==============================] - 332s 852ms/step - loss: 2.2841 - acc: 0.6418 - val_loss: 3.6350 - val_acc: 0.4517

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
390/390 [==============================] - 314s 805ms/step - loss: 2.3787 - acc: 0.6332 - val_loss: 3.3773 - val_acc: 0.4909
Epoch 7/35
390/390 [==============================] - 313s 803ms/step - loss: 2.1779 - acc: 0.6815 - val_loss: 3.5272 - val_acc: 0.4609
Epo

## Switched to Learning Rate Decay
---

## Total Epochs = 305

In [0]:
model = load_model(checkpoint_path)
print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.lr, 0.0005)
print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=35,
  max_queue_size=128 * 4,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')

Instructions for updating:
Use tf.cast instead.
Old Learning Rate: 0.0004428571555763483
New Learning Rate: 0.0005000000237487257
Epoch 1/35
390/390 [==============================] - 389s 999ms/step - loss: 1.7597 - acc: 0.7740 - val_loss: 3.0878 - val_acc: 0.5342
Epoch 2/35
390/390 [==============================] - 310s 794ms/step - loss: 1.7282 - acc: 0.7818 - val_loss: 3.0648 - val_acc: 0.5401
Epoch 3/35
390/390 [==============================] - 320s 822ms/step - loss: 1.7080 - acc: 0.7868 - val_loss: 3.0454 - val_acc: 0.5440
Epoch 4/35
390/390 [==============================] - 325s 834ms/step - loss: 1.6845 - acc: 0.7930 - val_loss: 3.0602 - val_acc: 0.5422
Epoch 5/35
390/390 [==============================] - 328s 840ms/step - loss: 1.6645 - acc: 0.7985 - val_loss: 3.0622 - val_acc: 0.5436

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/35
109/390 [=======>......................] - ETA: 3:42 - loss: 1.6520 - acc: 0.8021

## Colab crashed after just 5 epochs!
---
## Total epochs = 310

In [0]:
model = load_model(checkpoint_path)
print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.lr, 0.0005)
print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=30,
  max_queue_size=128 * 4,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')

Instructions for updating:
Use tf.cast instead.
Old Learning Rate: 0.0004428571555763483
New Learning Rate: 0.0005000000237487257
Epoch 1/30
390/390 [==============================] - 399s 1s/step - loss: 1.6510 - acc: 0.8014 - val_loss: 3.0383 - val_acc: 0.5471
Epoch 2/30
390/390 [==============================] - 313s 802ms/step - loss: 1.6337 - acc: 0.8072 - val_loss: 3.0499 - val_acc: 0.5472
Epoch 3/30
390/390 [==============================] - 334s 857ms/step - loss: 1.6212 - acc: 0.8091 - val_loss: 3.0344 - val_acc: 0.5509
Epoch 4/30
390/390 [==============================] - 334s 857ms/step - loss: 1.6101 - acc: 0.8109 - val_loss: 3.0470 - val_acc: 0.5501
Epoch 5/30
390/390 [==============================] - 334s 857ms/step - loss: 1.6021 - acc: 0.8132 - val_loss: 3.0469 - val_acc: 0.5482

Epoch 00005: saving model to ../checkpoints/check.ckpt
Epoch 6/30
390/390 [==============================] - 335s 859ms/step - loss: 1.5932 - acc: 0.8157 - val_loss: 3.0486 - val_acc: 0.5500
E

## Achieved 55.76 Validation Accuracy on Epoch 336

---
## Total Epochs = 340


In [0]:
model = load_model(checkpoint_path)
print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.lr, 0.000002)
print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.momentum, 0.99)

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=30,
  max_queue_size=128,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')

Old Learning Rate: 0.00037142858491279185
New Learning Rate: 1.9999999949504854e-06
Epoch 1/30
1562/1562 [==============================] - 632s 405ms/step - loss: 1.9404 - acc: 0.7078 - val_loss: 2.8872 - val_acc: 0.5665
Epoch 2/30
1562/1562 [==============================] - 580s 371ms/step - loss: 1.8846 - acc: 0.7210 - val_loss: 2.9150 - val_acc: 0.5613
Epoch 3/30
1562/1562 [==============================] - 615s 394ms/step - loss: 1.8518 - acc: 0.7263 - val_loss: 2.8752 - val_acc: 0.5668
Epoch 4/30
1562/1562 [==============================] - 618s 396ms/step - loss: 1.8309 - acc: 0.7327 - val_loss: 2.8589 - val_acc: 0.5713

Epoch 00004: saving model to ../checkpoints/check.ckpt
Epoch 5/30
1562/1562 [==============================] - 614s 393ms/step - loss: 1.8112 - acc: 0.7362 - val_loss: 2.8604 - val_acc: 0.5706
Epoch 6/30
1562/1562 [==============================] - 614s 393ms/step - loss: 1.8009 - acc: 0.7392 - val_loss: 2.8916 - val_acc: 0.5666
Epoch 7/30
1562/1562 [==========

## Achieved Validation Accuracy of 57.79 on Epoch 361

---

## Total epochs = 365

In [0]:
model = load_model(checkpoint_path)
print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.lr, 0.000002)
print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.momentum, 0.99)

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 128,
  validation_data=val_generator,
  validation_steps=10000 // 128,
  epochs=30,
  max_queue_size=128 * 2,
  callbacks=callbacks, verbose=1
)
model.save('model1.hdf5')

Old Learning Rate: 1.7714285149850184e-07
New Learning Rate: 1.9999999949504854e-06
Epoch 1/30
781/781 [==============================] - 480s 615ms/step - loss: 1.6299 - acc: 0.7820 - val_loss: 2.8289 - val_acc: 0.5747
Epoch 2/30
781/781 [==============================] - 417s 534ms/step - loss: 1.6321 - acc: 0.7814 - val_loss: 2.8591 - val_acc: 0.5722
Epoch 3/30
781/781 [==============================] - 455s 583ms/step - loss: 1.6360 - acc: 0.7792 - val_loss: 2.8841 - val_acc: 0.5675
Epoch 4/30
781/781 [==============================] - 456s 584ms/step - loss: 1.6325 - acc: 0.7813 - val_loss: 2.8603 - val_acc: 0.5694

Epoch 00004: saving model to ../checkpoints/check.ckpt
Epoch 5/30
781/781 [==============================] - 457s 585ms/step - loss: 1.6283 - acc: 0.7820 - val_loss: 2.8525 - val_acc: 0.5721
Epoch 6/30
781/781 [==============================] - 458s 586ms/step - loss: 1.6267 - acc: 0.7836 - val_loss: 2.8594 - val_acc: 0.5705
Epoch 7/30
781/781 [========================

## Achieved Validation Accuracy of 57.83 on Epoch 375

---

## Total Epochs = 395

In [0]:
model = load_model(checkpoint_path)
print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.lr, 2e-08)
print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.momentum, 0.99)

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 128,
  validation_data=val_generator,
  validation_steps=10000 // 128,
  epochs=10,
  max_queue_size=128 * 2,
  callbacks=callbacks, verbose=1
)

Old Learning Rate: 1.9428571818025375e-07
New Learning Rate: 1.999999987845058e-08
Epoch 1/10
781/781 [==============================] - 489s 625ms/step - loss: 1.6256 - acc: 0.7826 - val_loss: 2.8618 - val_acc: 0.5682
Epoch 2/10
781/781 [==============================] - 353s 451ms/step - loss: 1.6230 - acc: 0.7826 - val_loss: 2.8468 - val_acc: 0.5719

Epoch 00002: saving model to ../checkpoints/check.ckpt
Epoch 3/10
781/781 [==============================] - 457s 585ms/step - loss: 1.6255 - acc: 0.7824 - val_loss: 2.8791 - val_acc: 0.5673
Epoch 4/10
781/781 [==============================] - 458s 587ms/step - loss: 1.6312 - acc: 0.7813 - val_loss: 2.8730 - val_acc: 0.5676
Epoch 5/10
781/781 [==============================] - 458s 587ms/step - loss: 1.6284 - acc: 0.7815 - val_loss: 2.8486 - val_acc: 0.5732
Epoch 6/10
781/781 [==============================] - 457s 585ms/step - loss: 1.6220 - acc: 0.7823 - val_loss: 2.8460 - val_acc: 0.5739
Epoch 7/10
781/781 [=========================

## Achieved Validation Accuracy of 57.86 on Epoch 402

---

## Total Epochs = 405

### Tried this Hard Mining function but could'nt get it to work !


In [0]:
from os import environ
from keras.models import load_model
batch_size = 64
model = load_model(checkpoint_path)

def mine_hard_samples(model, datagen, batch_size):
   while True:
       samples, targets = [], []
       while len(samples) < batch_size:
           x_data, y_data = next(datagen)
           preds = model.predict(x_data)
           
           errors = np.abs(preds - y_data).max(axis=-1) > .99
           
           samples += x_data[errors].tolist()
           targets += y_data[errors].tolist()

       regular_samples = batch_size * 2 - len(samples)
       x_data, y_data = next(datagen)
       samples += x_data[:regular_samples].tolist()
       targets += y_data[:regular_samples].tolist()

       samples, targets = map(np.array, (samples, targets))

       idx = np.arange(batch_size * 2)
       np.random.shuffle(idx)
       batch1, batch2 = np.split(idx, 2)
       yield samples[batch1], targets[batch1]
       yield samples[batch2], targets[batch2]

def valid_main():
   model = load_model(checkpoint_path)

   x, y = next(train_generator)
   model.predict(x)

   model.fit_generator(mine_hard_samples(model, train_generator, 64),
                       steps_per_epoch=100000 // 64, 
                       epochs=1)

valid_main()

### List of references for easy lookup

---

1. Building blocks of interpretability: [Link](https://distill.pub/2018/building-blocks/) (Holy Grail of Intuition!)
2. Deep Residual Learning for image classification: [Link](https://arxiv.org/abs/1512.03385) (Resnet Paper)
3. Bag of tricks for image classification: [Link](https://arxiv.org/abs/1812.01187) (Tweaks and tricks to Resnet for increased performance paper)
2. Imbalanced Deep Learning by Minority Class
Incremental Rectification: [Link](https://arxiv.org/pdf/1804.10851.pdf) (Selectively Sampling Data paper)
2. Improved Regularization of Convolutional Neural Networks with Cutout: [Link](https://arxiv.org/pdf/1708.04552.pdf) (Cutout/Occlusion Augmentation paper)
3. Survey of resampling techniques for improving
classification performance in unbalanced datasets [Link](https://arxiv.org/pdf/1608.06048v1.pdf) (Resampling paper)
4. https://arxiv.org/pdf/1804.10851.pdf
5. https://github.com/wpriyadarshani/Resampling_Techniques
6.   https://www.researchgate.net/post/Machine_learning_if_proportion_of_number_of_cases_in_different_class_in_training_set_matters
7. https://arxiv.org/pdf/1801.05365.pdf
8. https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
9. https://www.analyticsvidhya.com/blog/2017/03/imbalanced-classification-problem/
10. https://arxiv.org/pdf/1608.06048v1.pdf